## Lab 6: Intrinsic Dimension and Density Estimation
You can use external libraries for linear algebra operations but you are expected to write your own algorithms.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA

# Exercise 1
Using the ```dry_beans_dataset``` as we did in previous laboratories (ie. follow the same proprocessing steps but **do not** perform the train-test split), program your own implementation of the two-NN estimate for the Intrinsic Dimension. 

Is the result comparible with what you would expect from an analysis of PCA's spectrum?

In [ ]:
df = pd.read_excel("../Datasets/Dry_Bean_Dataset.xlsx")

In [ ]:
df.head()

In [ ]:
y = df['Class']
X = df.drop('Class', axis=1)

In [ ]:
encoder = OrdinalEncoder()
y=np.array(y)
encoder.fit(y.reshape(-1,1))
y = encoder.transform(y.reshape(-1,1))
print(y)

In [ ]:
X = (X - X.mean())/X.std()

In [ ]:
X.head()

In [ ]:
pca = PCA()
pca.fit(X)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(pca.singular_values_)
plt.title("Eigenvalues spectrum")
plt.show()

In [ ]:
from dadapy.data import Data

In [ ]:
data = Data(np.array(X))

In [ ]:
id, _, r = data.compute_id_2NN()
#compute_id_2NN returns:
## id: estimated instrinsic dimension
## id_err: standard error on the id estimation
## rs: the average nearest neighbor distance

In [ ]:
print(f"Intrisic Dimension = {id}\nr = {r}")

Let us see what happens if we remove the instances for which the distance to their first NN is null.

**Note**: with ```sklearn.neighbors.NearestNeighbors``` the first NN is considered to be the point itself, so the first and second NNs are actually indexed by 1 and 2 (not 0 and 1).

In [ ]:
data.compute_distances(maxk=2)

In [ ]:
data.distances

In [ ]:
idx = np.where(data.distances[:,1]==0)
idx 
#as we said, it is only a small fraction of the dataset

In [ ]:
X = X.drop(idx[0])

In [ ]:
data = Data(np.array(X))

In [ ]:
id, _, r = data.compute_id_2NN()

In [ ]:
print(f"Intrisic Dimension = {id}\nr = {r}")

# Exercise 2
Using the following code, create a one-dimensional dataset of size $N=100$.

In [ ]:
from scipy.stats import norm, t #normal distribution, t distribution

np.random.seed(44)

In [ ]:
N = 100

X = np.concatenate(
    (np.random.standard_t(1, int(0.04*N))-3.5,np.random.normal(5, 1, int(0.48 * N)), np.random.normal(7.5, 1, int(0.48 * N)))
)[:, np.newaxis]

In [ ]:
X.shape

In [ ]:
X_plot = np.linspace(-12,12, 1000)[:, np.newaxis]
true_dens = 0.04* t(df=1,loc=-3.5).pdf(X_plot[:, 0]) + 0.48* norm(5, 1).pdf(X_plot[:, 0]) + 0.48*norm(7.5,1).pdf(X_plot[:,0])

fig = plt.figure(figsize=(10,8))
plt.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2)
plt.show()

Compute the density estimation with your implementations of:
- Histogram Density Estimation (Freedman-Diaconis rule)

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(15,10))

for i,bin in enumerate([2,5,12,24,50,100]):
    ax = axes[i//3, i%3]
    ax.hist(X, bins=bin)
    ax.set_title(f"Bins = {bin}")
plt.show()

In [ ]:
def friedman_diaconis(X):
    ''' 
    Select the number of bins such that bin=2*IQR(x)/(n^(1/3))
    '''
    q3, q1 = np.percentile(X, [75,25])
    iqr = q3 - q1
    return 2*iqr/(len(X)**(1/3))

In [ ]:
bin_fd=friedman_diaconis(X)
print(bin_fd)

In [ ]:
bin_number = int((max(X)-min(X))/bin_fd)
print(bin_number)

In [ ]:
plt.figure(figsize=(8,6))

plt.hist(X, bins=bin_number)
plt.title(f"Freedman Diaconis rule, bins = {bin_number}")
plt.show()

- Kernel Density Estimation (KDE) - Gaussian kernel (Silverman's rule)

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,12))

ax[0].fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="True distribution")
ax[1].fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="True distribution")


colors=['tab:orange', 'tab:red', 'tab:blue','tab:cyan']

for i,bandwidth in enumerate([0.1,0.5, 0.3, 0.8]):
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(X)
    log_dens = kde.score_samples(X_plot)
    ax[i%2].plot(X_plot[:,0], np.exp(log_dens), lw=3, linestyle="-", label=f"b={bandwidth}", c=colors[i])
ax[0].legend(loc='upper left')
ax[0].set_ylim(0, 0.3)
ax[1].legend(loc='upper left')
ax[1].set_ylim(0, 0.3)
plt.show()

In [ ]:
def silverman(X):
    '''
    There is no built in function to compute the bandwidth with Silverman's rule in sklearn's framework 
    (see scipy.stat.gaussian_kde(dataset, bw_method='silverman') for an alternative)
    
    h = 0.9 * min(sigma, IQR/1.34)*n^(-1/5)
    '''
    
    q3, q1 = np.percentile(X, [75,25])
    iqr = q3 - q1
    sigma = np.std(X)

    return 0.9*min(sigma, iqr/1.34)*len(X)**(-1/5)

In [ ]:
silverman_bandwidth = silverman(X)
print(silverman_bandwidth)

In [ ]:
plt.figure(figsize=(15,10))

plt.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="True distribution")

kde = KernelDensity(kernel='gaussian', bandwidth=silverman_bandwidth).fit(X)
log_dens = kde.score_samples(X_plot)
plt.plot(X_plot[:,0], np.exp(log_dens), color="black", lw=2, linestyle="-", label="Gaussian kernel")
plt.legend(loc='upper left')


plt.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), "+k")
plt.show()